## Data Preprocessing

In [2]:
import numpy as np
import pandas as pd
from io import StringIO

### Sample data

In [3]:
csv_data = '''A,B,C,D
    1.0,2.0,3.0,4.0
    5.0,6.0,,8.0
    0.0,11.0,12.0
    11.0,21.0,,'''
csv_data=unicode(csv_data)
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,0.0,11.0,12.0,NaN
3,11.0,21.0,NaN,NaN


### Handling missing values

In [4]:
#Counting number of missing values along row/column
print(df.isnull())
df.isnull().sum()

       A      B      C      D
0  False  False  False  False
1  False  False   True  False
2  False  False  False   True
3  False  False   True   True


A    0
B    0
C    2
D    2
dtype: int64

### Eliminating rows/columns with missing data

In [5]:
#Eliminates rows which contain missing data
df.dropna(axis=0) 
#axis =1 eliminated columns containing missing data

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [6]:
# Keeps only rows which have atleast 3 non-NaN values
df.dropna(thresh=3) 

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,0.0,11.0,12.0,NaN


In [7]:
# To drop rows where missing values in a particular column
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,0.0,11.0,12.0,NaN


In [8]:
# To drop rows whose entire values are missing 
#Note: In this case there is no such row with all data missing
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,0.0,11.0,12.0,NaN
3,11.0,21.0,NaN,NaN


### Imputing missing values

In [9]:
# Replace missing value by the mean value of the entire feature column.
#Note: only number of rows with data are considered for mean calculation

from sklearn.preprocessing import Imputer
imr = Imputer(missing_values='NaN', strategy='mean', axis=0)
imr = imr.fit(df)
newdf = imr.transform(df.values)
newdf

array([[  1. ,   2. ,   3. ,   4. ],
       [  5. ,   6. ,   7.5,   8. ],
       [  0. ,  11. ,  12. ,   6. ],
       [ 11. ,  21. ,   7.5,   6. ]])

### Handling Categorical data

In [10]:
df = pd.DataFrame([
                    ['green', 'M', 10.1, 'class1'],
                    ['red', 'L', 13.5, 'class2'],
                    ['blue', 'XL', 15.3, 'class2'],
                    ['yellow', 'X', 12.3, 'class1']
                  ])
df.columns = ['color', 'size', 'price', 'classlabel']
#numerical or nominal/ordinal features 
#nominal - cannot be ordered in a particular order eg: color column in table below
# ordinal - features which can be sorted eg: size column in table below
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class2
3,yellow,X,12.3,class1


In [11]:
#Mapping ordinal featues to numerical.
size_mapping = {'M':1, "L":2, "X":3, "XL":4}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,4,15.3,class2
3,yellow,3,12.3,class1


In [12]:
#inverse mapping
inv_size_mapping = {v:k for k,v in size_mapping.items()}
inv_size_mapping

{1: 'M', 2: 'L', 3: 'X', 4: 'XL'}

### Encoding class labels

In [13]:
class_mapping = {label:idx for idx,label in enumerate(np.unique(df['classlabel']))}
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,4,15.3,class2
3,yellow,3,12.3,class1


In [14]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,4,15.3,1
3,yellow,3,12.3,0


### Encoding class labels - using python library

In [15]:
from sklearn.preprocessing import LabelEncoder
df = pd.DataFrame([
                    ['green', 'M', 10.1, 'class1'],
                    ['red', 'L', 13.5, 'class2'],
                    ['blue', 'XL', 15.3, 'class2'],
                    ['yellow', 'X', 12.3, 'class1']
                  ])
df.columns = ['color', 'size', 'price', 'classlabel']

classle = LabelEncoder()
df['classlabel'] = classle.fit_transform(df['classlabel'])
df['color'] = classle.fit_transform(df['color'])
df

,color,size,price,classlabel
0,1,M,10.1,0
1,2,L,13.5,1
2,0,XL,15.3,1
3,3,X,12.3,0


### One hot encoding using get_dummies

In [16]:
df = pd.DataFrame([
                    ['green', 'M', 10.1, 'class1'],
                    ['red', 'L', 13.5, 'class2'],
                    ['blue', 'XL', 15.3, 'class2'],
                    ['yellow', 'X', 12.3, 'class1']
                  ])
df.columns = ['color', 'size', 'price', 'classlabel']

In [17]:
df1=pd.get_dummies(df['color'])
df = pd.concat([df,df1],axis=1)
df

,color,size,price,classlabel,blue,green,red,yellow
0,green,M,10.1,class1,0,1,0,0
1,red,L,13.5,class2,0,0,1,0
2,blue,XL,15.3,class2,1,0,0,0
3,yellow,X,12.3,class1,0,0,0,1


### Sort by columns

In [18]:
df.sort_values(by = ['color','size'],axis=0)

,color,size,price,classlabel,blue,green,red,yellow
2,blue,XL,15.3,class2,1,0,0,0
0,green,M,10.1,class1,0,1,0,0
1,red,L,13.5,class2,0,0,1,0
3,yellow,X,12.3,class1,0,0,0,1


### Changing the datatype

In [20]:
df.iloc[:,4:5].astype(float)

,blue
0,0.0
1,0.0
2,1.0
3,0.0
